In [2]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import ipywidgets as widgets
import serial
import time
from picoscope import ps5000a
import picosdk
from picosdk.discover import find_all_units
import serial.tools.list_ports as port_list
import chipwhisperer as cw

# Check if the Picoscope is connected
scopes = find_all_units()
for scope in scopes:
    print("Working with:")
    print(scope.info)
    scope.close()
ports = list(port_list.comports())
for p in ports:
    print (p)

ModuleNotFoundError: No module named 'chipwhisperer'

**PicoScope setup**

In [ ]:
ps = ps5000a.PS5000a()
print("Found the following picoscope:")
print(ps.getAllUnitInfo())

# Since target runnning at 10 MHz and AES requires from trigger
obs_duration = 3.225E-6
# Sample at least 1260 points within that window
sampling_interval = obs_duration / 1260
# Configure timebase
(actualSamplingInterval, nSamples, maxSamples) = ps.setSamplingInterval(sampling_interval, obs_duration)
print("Nsamples : ", nSamples)
print("Sampling interval = %f us" % (actualSamplingInterval*nSamples*1E6))

# 50mV range on channel A, AC coupled, 20 MHz BW limit
ps.setChannel('A', 'AC', 0.05, 0.0, enabled=True, BWLimited=True)
# Channel B is trigger
ps.setChannel('B', 'DC', 10.0, 0.0, enabled=True)
ps.setSimpleTrigger('B', 2.0, 'Rising', timeout_ms=2000, enabled=True)

**CW305 setup**

In [ ]:
sbox_dict = {
    "sbox_aes" : "RIJANDAEL",
    "sbox_freyre_1" : "FREYRE_1",
    "sbox_freyre_2" : "FREYRE_2",
    "sbox_freyre_3" : "FREYRE_3",
    "sbox_hussain_6" : "HUSSAIN_6",
    "sbox_ozkaynak_1" : "OZKAYNAK_1",
    "sbox_azam_1" : "AZAM_1",
    "sbox_azam_2" : "AZAM_2",
    "sbox_azam_3" : "AZAM_3"
}

sbox_type = "sbox_freyre_1"

In [ ]:
bitstream = r"../rtl/fpga/bitstream/aes/cw305_top_aes_single_round_" + sbox_dict[sbox_type] + r".bit"
target = cw.target(scope, cw.targets.CW305, bsfile=bitstream, force=True)

print(target.is_programmed())
print(bitstream)

target.vccint_set(1.0)
target.pll.pll_enable_set(True)         # enable PLL chip
target.pll.pll_outenable_set(False, 0)  # disable PLL 0
target.pll.pll_outenable_set(True, 1)   # enable PLL 1
target.pll.pll_outenable_set(False, 2)  # disable PLL 2
target.pll.pll_outfreq_set(10E6, 1)     # PLL1 frequency set to 10 MHz
# 1 ms is plenty idling time --> maybe not useful with picoscope (?) 
target.clksleeptime = 1 

**Capture loop**

In [ ]:
def pico_capture():
    # Arm the picoscope
    ps.runBlock()
    time.sleep(0.05)
    # Trigger the encryption on Target
    target.fpga_write(target.REG_USER_LED, [0x01])
    target.usb_trigger_toggle()
    ps.waitReady()
    # Capture the trace 
    data = ps.getDataV('A', nSamples, returnOverflow=False)
    return data

In [ ]:
from tqdm.notebook import tnrange
#project = cw.open_project(project_file)
from Crypto.Cipher import AES
from chipwhisperer.common.traces import Trace
from AES import AES as AESpy

max_iteration = 20

for iteration in range(1,max_iteration+1):
    print("Iteration", iteration)
    project_file = "../build/sca_success_rate_" + sbox_dict[sbox_type] + "/sca_test_CW305_pop_" + str(iteration) + ".cwp"
    project = cw.create_project(project_file, overwrite=True)
    
    ktp = cw.ktp.Basic()
    # Initialize cipher to verify DUT result:
    key, text = ktp.next()
    cipher = AES.new(bytes(key), AES.MODE_ECB)
    print("Key: ", [ hex(subkey) for subkey in key])

    N = 40000        # Number of traces
    traces = []
    textin = []
    keys = []
    data_mV = []

    target.fpga_write(target.REG_CRYPT_KEY, key[::-1])

    for i in range((iteration-1)*50000):
        key, text = ktp.next()

    # Dummy capture call due to bug of using AC coupling
    pico_capture()

    for i in tnrange(N, desc='Capturing traces'):

        # Write plaintext to target
        inputtext = text[::-1]
        target.fpga_write(target.REG_CRYPT_TEXTIN, inputtext)

        # Capture the trace 
        data = pico_capture()
        traces.append(np.array(data))
        data_mV.append(np.array(data)*1E3)

        # Organize and store data
        response = target.fpga_read(target.REG_CRYPT_CIPHEROUT, 16)
        response = response[::-1]
        trace_i = Trace(np.array(data), text, response, key)
        project.traces.append(trace_i)

        # Sanity check with expected ciphertext
        #print("Response fpga: ",[ hex(el) for el in response])
        #print("Expected instead: ", [hex(el) for el in cipher.encrypt(bytes(text))])
        key_py = str(key).replace(" ", "")
        key_py = key_py[14:-2]

        text_py = str(text).replace(" ", "")
        text_py = text_py[14:-2]
        text_py = [text_py[i:i+2] for i in range(0, len(text_py), 2)]
        text_py = ['0x' + element for element in text_py]
        text_py = [int(s, 16) for s in text_py]

        ciphertext = AESpy.encrypt(key_py, text_py, sbox_type)
        assert (list(response) == ciphertext), "Incorrect encryption result!\nGot {}\nExp {}\n".format(list(response), ciphertext)

        key, text = ktp.next() 
        textin.append(text)
        keys.append(key)

    project.save()
    project.close()
target.dis()

**Success rate**

In [2]:
import chipwhisperer.analyzer as cwa
from chipwhisperer.common.traces import Trace
import chipwhisperer as cw
from IPython.display import clear_output
import pandas as pd
import numpy as np
from AES import AES as AESpy

class AES128_Round10_Model(cwa.AESLeakageHelper):
    def leakage(self, pt, ct, guess, bnum):
        """
        Args:
            pt: 16-byte plain-text input
            ct: 16-byte cipher-text output.
            key: 16-byte AES key - byte 'bnum' may be a GUESS if key is known. Rest of bytes may/may not be valid too.
            bnum: Byte number we are trying to attack.
        """
        inv_sbox_m = { "sbox_aes" : [
        0x52,0x09,0x6a,0xd5,0x30,0x36,0xa5,0x38,0xbf,0x40,0xa3,0x9e,0x81,0xf3,0xd7,0xfb,
        0x7c,0xe3,0x39,0x82,0x9b,0x2f,0xff,0x87,0x34,0x8e,0x43,0x44,0xc4,0xde,0xe9,0xcb,
        0x54,0x7b,0x94,0x32,0xa6,0xc2,0x23,0x3d,0xee,0x4c,0x95,0x0b,0x42,0xfa,0xc3,0x4e,
        0x08,0x2e,0xa1,0x66,0x28,0xd9,0x24,0xb2,0x76,0x5b,0xa2,0x49,0x6d,0x8b,0xd1,0x25,
        0x72,0xf8,0xf6,0x64,0x86,0x68,0x98,0x16,0xd4,0xa4,0x5c,0xcc,0x5d,0x65,0xb6,0x92,
        0x6c,0x70,0x48,0x50,0xfd,0xed,0xb9,0xda,0x5e,0x15,0x46,0x57,0xa7,0x8d,0x9d,0x84,
        0x90,0xd8,0xab,0x00,0x8c,0xbc,0xd3,0x0a,0xf7,0xe4,0x58,0x05,0xb8,0xb3,0x45,0x06,
        0xd0,0x2c,0x1e,0x8f,0xca,0x3f,0x0f,0x02,0xc1,0xaf,0xbd,0x03,0x01,0x13,0x8a,0x6b,
        0x3a,0x91,0x11,0x41,0x4f,0x67,0xdc,0xea,0x97,0xf2,0xcf,0xce,0xf0,0xb4,0xe6,0x73,
        0x96,0xac,0x74,0x22,0xe7,0xad,0x35,0x85,0xe2,0xf9,0x37,0xe8,0x1c,0x75,0xdf,0x6e,
        0x47,0xf1,0x1a,0x71,0x1d,0x29,0xc5,0x89,0x6f,0xb7,0x62,0x0e,0xaa,0x18,0xbe,0x1b,
        0xfc,0x56,0x3e,0x4b,0xc6,0xd2,0x79,0x20,0x9a,0xdb,0xc0,0xfe,0x78,0xcd,0x5a,0xf4,
        0x1f,0xdd,0xa8,0x33,0x88,0x07,0xc7,0x31,0xb1,0x12,0x10,0x59,0x27,0x80,0xec,0x5f,
        0x60,0x51,0x7f,0xa9,0x19,0xb5,0x4a,0x0d,0x2d,0xe5,0x7a,0x9f,0x93,0xc9,0x9c,0xef,
        0xa0,0xe0,0x3b,0x4d,0xae,0x2a,0xf5,0xb0,0xc8,0xeb,0xbb,0x3c,0x83,0x53,0x99,0x61,
        0x17,0x2b,0x04,0x7e,0xba,0x77,0xd6,0x26,0xe1,0x69,0x14,0x63,0x55,0x21,0x0c,0x7d],
                    "sbox_freyre_1" : [
        0x4b,0xd5,0xdb,0x5b,0x7a,0x77,0xa4,0x15,0x0b,0xc6,0xf9,0x7c,0xe4,0x08,0x58,0xb3,
        0xf7,0x18,0x27,0x5e,0xb9,0xfe,0xcf,0x42,0x64,0x1b,0xc0,0xf1,0x81,0x32,0x1e,0x87,
        0x16,0xba,0x67,0xf4,0xe9,0xc5,0x6a,0xde,0x86,0x85,0x72,0xf0,0xa9,0x3d,0x8f,0xf3,
        0x79,0xec,0x98,0x70,0x96,0x01,0xe1,0xb8,0xdd,0xa0,0x11,0xab,0x02,0x5a,0x19,0xa6,
        0x45,0xc8,0xea,0x24,0x56,0xe3,0x8b,0x22,0x55,0xaa,0xa2,0x23,0x37,0xb1,0x8c,0xf2,
        0x74,0xcb,0xd6,0xff,0xfa,0x13,0xbd,0x4a,0x9b,0x0f,0x93,0xd4,0x7f,0x91,0x6d,0x84,
        0xed,0x88,0x6f,0x52,0xce,0x92,0x9e,0xc2,0xc1,0x9c,0x90,0x44,0x31,0x8d,0xa5,0x57,
        0x51,0x83,0xcc,0x9a,0x80,0xe5,0xef,0xda,0x62,0x0e,0x82,0xe6,0x35,0x54,0x68,0xbb,
        0x3a,0x69,0xe7,0x6e,0xb7,0x10,0x39,0x7e,0x95,0x46,0x3c,0x0d,0x06,0xdf,0xbe,0xbc,
        0x29,0x20,0xb4,0x2d,0x61,0x40,0x3f,0x66,0xbf,0xa3,0xee,0x41,0x71,0xc4,0x2c,0x65,
        0x34,0x2f,0x2a,0x4d,0x50,0x21,0x03,0x09,0x60,0x5d,0x63,0xfd,0xe2,0xc9,0xb6,0x59,
        0xdc,0x8e,0xb2,0xf8,0xd3,0x5c,0x3e,0xb5,0x33,0x6c,0xd1,0x4e,0xeb,0x28,0x2b,0x07,
        0x05,0xd8,0x9d,0x7d,0xa7,0xd2,0x30,0x26,0xac,0x1f,0x12,0xc7,0xd0,0x75,0xf6,0xca,
        0x48,0xb0,0xa1,0x7b,0xad,0x5f,0x94,0x38,0x9f,0xaf,0x89,0x25,0x1d,0x49,0x14,0x8a,
        0x4c,0xfc,0x43,0xcd,0x0c,0x04,0xe0,0xa8,0x1c,0xc3,0x73,0xe8,0x0a,0x47,0x76,0xf5,
        0xae,0x53,0x00,0x97,0x4f,0xd7,0x99,0xfb,0x2e,0x3b,0x17,0x36,0x6b,0xd9,0x78,0x1a],
                    "sbox_freyre_2" : [
        0xf2,0x42,0x09,0x8a,0x9b,0xac,0x7a,0x32,0x6b,0xfd,0xf9,0x85,0xa8,0xeb,0xd3,0x4a,
        0x60,0x20,0xb2,0xec,0x31,0xf6,0x90,0x7d,0xca,0x4d,0xba,0x73,0x4b,0x39,0xdd,0x3c,
        0xe3,0x94,0x81,0x99,0xc1,0xb0,0x8e,0x2d,0x9f,0xb6,0x3e,0x5b,0x75,0xe0,0x93,0x3b,
        0xd0,0xce,0x89,0x5a,0x10,0xee,0xc6,0x5f,0x01,0xa1,0xa7,0x61,0xfe,0x05,0x36,0xad,
        0x2b,0xb9,0xdb,0xd2,0x51,0xdf,0xea,0x27,0x58,0xc3,0x1b,0x2c,0x0b,0x83,0x92,0xbe,
        0x18,0xbd,0x6f,0xa0,0x2f,0x00,0x88,0x66,0xaa,0x7c,0x24,0xaf,0x04,0xa9,0x8b,0xb7,
        0x46,0xb1,0x50,0x37,0xe8,0x33,0x77,0x19,0x5c,0x1a,0x69,0x7b,0x55,0xd6,0x52,0x43,
        0xfa,0xf1,0x4f,0x54,0x4e,0xb8,0xed,0xe9,0x3d,0x97,0x70,0x8f,0xa5,0x34,0x3f,0x9e,
        0x53,0xe7,0x3a,0x7e,0x02,0x82,0x17,0xf4,0x71,0x35,0x68,0xc7,0x13,0x6c,0x5e,0xd7,
        0x1c,0x78,0xab,0xbf,0x11,0x29,0x63,0xc0,0xbb,0xd9,0x79,0x03,0xe4,0x12,0x30,0xf7,
        0xa3,0x98,0xe1,0xb4,0x49,0xf5,0x62,0xd1,0xd4,0x38,0xe5,0xf8,0x96,0x86,0x15,0x21,
        0x72,0x28,0x6d,0x8c,0x1f,0x44,0x59,0xc4,0xc8,0x26,0xff,0x1d,0xb3,0x6e,0x2e,0x0c,
        0x06,0x64,0x40,0x0a,0x0d,0x41,0x9c,0x0f,0x57,0xcc,0xfb,0xa2,0xd8,0x91,0x14,0x65,
        0xf0,0xef,0x1e,0xdc,0xa4,0x80,0xae,0x74,0xc9,0x2a,0x45,0x7f,0x8d,0xe2,0xbc,0x4c,
        0xc5,0xb5,0xc2,0x6a,0x9d,0xcd,0x76,0x48,0x23,0x0e,0x67,0x47,0x9a,0x16,0xda,0xe6,
        0x22,0xcb,0x08,0xf3,0x87,0xcf,0x07,0xd5,0xde,0x56,0xfc,0xa6,0x5d,0x25,0x84,0x95],
                    "sbox_freyre_3" : [
        0xd0,0x79,0x12,0xd2,0x56,0x04,0x83,0xff,0x7b,0x33,0xf9,0xa8,0xae,0x06,0x95,0x9a,
        0x2a,0xd1,0xac,0x14,0x3d,0xec,0xbe,0x32,0x41,0xb7,0x7d,0x40,0x78,0x9b,0x73,0x5c,
        0xb8,0xd5,0xad,0x60,0x39,0xd6,0xed,0x35,0x10,0x2c,0xb9,0xa5,0x58,0x61,0x4f,0x7f,
        0xd7,0x6b,0xc0,0x1b,0x26,0x01,0x98,0xc3,0x80,0x17,0x18,0x16,0x3b,0xe1,0x38,0x34,
        0xd4,0x8f,0x3f,0x42,0x54,0x6e,0x6f,0xe4,0x29,0x3c,0x50,0x0a,0x7c,0x9c,0x59,0x5d,
        0xd3,0x57,0xb6,0xaf,0x25,0xdf,0x90,0x37,0xc6,0x52,0xb2,0x44,0xe5,0x8d,0xd9,0xcd,
        0xea,0x09,0x94,0x91,0xf3,0x03,0x28,0xa6,0xc2,0x87,0x43,0x75,0x3a,0x7e,0xc8,0x23,
        0xde,0xc9,0xc7,0xa1,0x6c,0x86,0x30,0x0e,0x99,0xb4,0x22,0xce,0xf0,0x71,0x88,0x77,
        0x97,0xba,0x82,0xfe,0xbb,0x81,0xe8,0x24,0x02,0x13,0x4d,0xb1,0x92,0xaa,0x3e,0x89,
        0x47,0x07,0x11,0x45,0x1d,0x19,0x53,0xe7,0x2e,0x08,0x93,0x5f,0x5a,0x2d,0xe3,0x8b,
        0xfc,0xfa,0xf2,0xa0,0xc5,0xca,0x69,0xab,0x31,0x4c,0x2b,0xb5,0xa2,0x66,0x51,0xa3,
        0x15,0x8a,0x8c,0x6d,0xdd,0x1e,0x70,0xdb,0x1c,0x1a,0x8e,0x5e,0xf4,0xf8,0xf1,0x21,
        0xeb,0x05,0x96,0xcb,0xe6,0x55,0x63,0x20,0x46,0xfd,0x65,0x67,0x00,0xb3,0xe9,0x64,
        0xc4,0xee,0xa4,0x62,0x85,0x36,0xe2,0xa9,0xef,0x2f,0x1f,0xcf,0xbd,0xf6,0x9e,0xe0,
        0x68,0xc1,0xfb,0xbf,0x84,0xdc,0x4b,0x9f,0xbc,0xd8,0x76,0x74,0x0d,0xcc,0x72,0xf5,
        0x7a,0xda,0x5b,0x4a,0xb0,0xf7,0x27,0xa7,0x0b,0x4e,0x0f,0x0c,0x49,0x9d,0x6a,0x48],
                    "sbox_hussain_6" : [
        0x25,0x03,0xfb,0x18,0xa3,0xf4,0x79,0x0f,0x27,0x68,0x4d,0xd5,0xf1,0x88,0x1b,0x72,
        0x45,0x6a,0xee,0x24,0x74,0xd1,0xf6,0xaa,0x09,0xc8,0x7f,0x8b,0x0a,0xea,0x8c,0x38,
        0xa6,0x5a,0x3a,0x7d,0x11,0xcd,0xfc,0x6e,0x6f,0x89,0xc5,0x6b,0x8a,0x7c,0x01,0xce,
        0x49,0x32,0x59,0x3c,0xa1,0x5c,0x26,0xeb,0xba,0x3d,0x85,0x54,0xb3,0x5f,0xbb,0x99,
        0x90,0x63,0x04,0x3f,0x19,0x76,0x2f,0x4b,0x8f,0x86,0x4a,0x56,0xdd,0x70,0x21,0x08,
        0xb7,0x2c,0xc4,0xaf,0x1f,0x92,0x7b,0x81,0x8d,0xcb,0x4e,0x3b,0x0e,0xa2,0xbe,0x35,
        0x5d,0x83,0xe9,0xbf,0x14,0xae,0xec,0x37,0x46,0x61,0xcc,0x29,0x33,0x13,0x95,0xf5,
        0x23,0xd3,0xb0,0xe4,0x17,0x98,0xe0,0xb1,0x00,0xa0,0x50,0x60,0xd4,0x0b,0xc3,0x16,
        0xc7,0xb9,0x43,0x4c,0x69,0x7e,0x2e,0xc1,0xc6,0xdf,0xe5,0x7a,0x06,0x91,0xd2,0x39,
        0x80,0x82,0x67,0xd6,0xcf,0x6c,0xa9,0x87,0x0d,0xac,0x1c,0x10,0x71,0xab,0x1e,0x55,
        0x93,0xff,0x34,0x9e,0x47,0x53,0x73,0xfd,0x84,0x9b,0x64,0xd9,0xf7,0xa4,0xa7,0xc9,
        0x1a,0x42,0x2a,0x1d,0xe3,0x97,0x0c,0xa5,0xe1,0xf0,0xdb,0xa8,0x40,0x6d,0x41,0xc0,
        0x3e,0xf2,0xe8,0xed,0xca,0xda,0x9f,0x57,0x07,0xde,0x51,0xd8,0x15,0x5b,0x5e,0x9c,
        0xe2,0x9d,0xb5,0xb8,0x65,0x2d,0xbd,0xdc,0xc2,0xf3,0xfa,0x31,0xb2,0x77,0x96,0x12,
        0x28,0x78,0x44,0xbc,0x22,0x58,0x8e,0xe6,0x05,0xad,0x75,0x48,0x9a,0xd0,0xf9,0xfe,
        0x66,0x2b,0x94,0x52,0xb4,0x02,0xef,0x4f,0xf8,0x20,0x30,0xb6,0xe7,0x62,0x36,0xd7],
                    "sbox_ozkaynak_1" : [
        0xe6,0x23,0x24,0x45,0x6a,0x49,0xf6,0x31,0xc1,0x93,0x35,0xce,0x87,0xfb,0xb7,0x14,
        0xf1,0xfe,0x83,0xec,0xf4,0x38,0xbc,0xd1,0xb1,0xbb,0x39,0x59,0x77,0x2b,0x9f,0x80,
        0x33,0xd8,0x65,0x41,0xbf,0xc6,0xff,0x42,0x0b,0xbe,0x7a,0x67,0x26,0x32,0xdf,0xa3,
        0x6c,0x9a,0x9c,0x89,0x29,0x36,0x95,0x16,0xdb,0x0e,0xae,0x63,0x98,0x19,0x15,0x90,
        0x7f,0x9d,0xe0,0x43,0x5a,0x2e,0xa8,0x61,0x27,0xd2,0x1c,0xbd,0x37,0xe1,0xda,0xcf,
        0xf7,0x0f,0xb6,0x1a,0x7c,0x0c,0xd5,0x75,0x13,0xf9,0x07,0x52,0xef,0x85,0xa9,0xf8,
        0x55,0x51,0x3e,0x00,0x82,0x7b,0x5f,0xeb,0xd4,0x17,0xb0,0x3b,0x05,0x68,0x44,0x9e,
        0x4b,0x46,0x79,0xa4,0x18,0xfa,0x1e,0xe4,0x88,0xca,0x69,0x53,0x58,0x48,0x22,0x78,
        0xba,0x5e,0x04,0xdc,0x8f,0xd9,0x21,0x1d,0x7d,0xb5,0xc7,0xc0,0xc4,0x5d,0x94,0xa5,
        0x6d,0x97,0x3c,0x76,0xab,0x10,0x91,0xa1,0x03,0xfc,0xb9,0x4c,0x72,0xb4,0x6f,0x02,
        0xa6,0x01,0xd3,0x4e,0xc3,0x4a,0xd6,0xd0,0x54,0x3f,0x47,0x3d,0x2d,0xf0,0xb3,0x3a,
        0xe5,0xcd,0xd7,0xe2,0xac,0xa2,0xc8,0xde,0x84,0x99,0xaa,0x34,0x74,0xad,0xdd,0xc9,
        0x8b,0x96,0x0a,0xf2,0xe9,0xf3,0x4d,0x2a,0x40,0xed,0x8e,0xe8,0x0d,0x30,0x11,0xf5,
        0xb2,0x7e,0x9b,0x66,0x5b,0x6b,0x12,0x62,0xcc,0x70,0x1f,0xb8,0x8a,0xe3,0x56,0x60,
        0xe7,0x1b,0x2c,0x25,0x6e,0x28,0xea,0x86,0x71,0xee,0x06,0xc2,0x4f,0xa0,0x2f,0x64,
        0x09,0x57,0x73,0xfd,0x8c,0x8d,0xa7,0xaf,0xc5,0x20,0x92,0x5c,0x08,0x81,0x50,0xcb],
                    "sbox_azam_1" : [
        0x9d,0x4b,0x13,0x0d,0x50,0xd9,0x1b,0x15,0x68,0xa9,0x23,0xc0,0x3b,0xd4,0x9b,0xb3,
        0xfe,0xbd,0xef,0x41,0xa2,0x09,0xcd,0x6e,0x4f,0xdc,0x7f,0xd5,0x1d,0x05,0x58,0xe4,
        0xfd,0x2f,0x54,0xb8,0x8a,0x07,0x8f,0xab,0x0f,0x79,0xf3,0x59,0x1e,0xc3,0xf5,0x40,
        0xf7,0x6c,0x8b,0x44,0x11,0x21,0x12,0x48,0x46,0x5c,0x70,0xdb,0x65,0x81,0x94,0x64,
        0xc9,0xbe,0xee,0x1f,0x08,0xd1,0x75,0xe3,0x32,0x14,0x90,0xa4,0x5e,0xe1,0x0b,0xde,
        0xd8,0x2c,0x87,0x73,0x8c,0x04,0xe7,0x4a,0x8e,0xeb,0x83,0x0a,0x8d,0xb1,0x77,0xe6,
        0x30,0x7c,0xea,0x6b,0xd0,0xb2,0x71,0x2d,0x43,0xe5,0x22,0x7e,0xfb,0x80,0x03,0xf6,
        0xcc,0x53,0xae,0xe2,0x3f,0x4d,0x99,0xc8,0xa7,0xc6,0x74,0xa0,0x98,0xc1,0xc7,0x78,
        0x96,0x42,0x5a,0x25,0xb6,0x47,0x33,0xfa,0x91,0x5d,0x92,0x6a,0x36,0xc5,0xaf,0x35,
        0xca,0x38,0x1a,0x86,0x0e,0xa5,0xe9,0xdd,0xd6,0xfc,0x00,0xdf,0x29,0x26,0x88,0x31,
        0xa8,0x2b,0xf0,0xec,0x34,0xce,0x67,0x85,0x16,0x9a,0xc4,0x56,0x1c,0x62,0x51,0x52,
        0x95,0xb5,0xc2,0xbc,0xac,0x9f,0xcb,0xb0,0x19,0x27,0xbf,0x28,0x3d,0x97,0x69,0x5b,
        0x84,0x37,0x7a,0x20,0xda,0x3e,0x10,0x06,0x6d,0x17,0xa3,0x4c,0x24,0xff,0x2a,0xad,
        0x0c,0xe8,0x4e,0x2e,0xb7,0x55,0xf1,0xf8,0xd3,0x01,0xf2,0xa6,0x49,0x3c,0xcf,0x63,
        0xf9,0x89,0x72,0x02,0x9c,0x18,0x66,0x39,0x93,0x7b,0x5f,0xd7,0x45,0x3a,0xb4,0xba,
        0xe0,0x76,0xf4,0x9e,0x61,0x6f,0xed,0x60,0x82,0xbb,0xb9,0x7d,0xd2,0xaa,0xa1,0x57],
                    "sbox_azam_2" : [
        0x70,0x0d,0xc5,0x61,0x90,0x39,0x25,0xfe,0x92,0x1d,0x8a,0x54,0x04,0x31,0xd2,0xf2,
        0xc3,0xb7,0x43,0x24,0x75,0x0c,0xe0,0x79,0xc2,0x4e,0x7a,0x56,0x68,0xa9,0x64,0xb4,
        0xc6,0x00,0x59,0x7b,0x60,0xba,0xac,0xe4,0x93,0x1b,0x12,0x45,0xe1,0x87,0x1c,0x30,
        0x29,0x78,0x88,0xde,0x4a,0x9e,0x2c,0x3f,0x7f,0x23,0xf3,0xf0,0x40,0x10,0x77,0x14,
        0xd8,0x02,0xa8,0x8f,0x98,0x28,0x84,0x86,0x94,0x16,0x6c,0xcb,0xd4,0x50,0x3a,0xd7,
        0x3b,0x2f,0x0a,0x99,0xfd,0xd5,0xe9,0x8b,0x32,0xd1,0x2e,0x4c,0x96,0x8d,0x27,0x5c,
        0x06,0xbd,0xd3,0xe3,0x26,0x7d,0xb2,0x05,0x5a,0x41,0xfa,0xa7,0xe8,0x1e,0x69,0xad,
        0x71,0x6f,0x97,0x4b,0x2a,0xb9,0x82,0xbe,0x91,0x52,0x53,0x07,0x20,0x95,0x09,0xb5,
        0xf5,0x35,0x81,0xa6,0xea,0x5e,0xc1,0x48,0xa2,0x46,0x18,0xdb,0xf7,0x62,0xcf,0x9d,
        0x73,0x3e,0x1a,0xd6,0x7c,0x83,0x3c,0x01,0xdc,0x17,0xc9,0x0b,0x76,0x57,0x9c,0xfc,
        0xa4,0x6b,0x65,0xec,0xbc,0xfb,0x49,0xc0,0x19,0xe5,0x51,0xb0,0xce,0x85,0xf6,0x58,
        0x5f,0x8e,0x15,0x13,0xd9,0x80,0x6d,0xb8,0x1f,0xdf,0x0f,0xca,0xab,0xe7,0x08,0xf9,
        0xf8,0x67,0xe2,0xa3,0x9f,0x33,0x37,0x4f,0x3d,0x5b,0xf1,0xe6,0x89,0xae,0x36,0x03,
        0x4d,0xee,0x44,0xbb,0x42,0xa5,0xda,0xc7,0x2b,0xb6,0x63,0xbf,0x66,0x38,0x5d,0x6a,
        0x11,0x9a,0x6e,0xc8,0x2d,0x0e,0xaa,0xcc,0x8c,0x7e,0x72,0xb3,0x22,0xeb,0xdd,0xed,
        0xcd,0x74,0xa1,0x21,0xd0,0xef,0x9b,0x34,0xf4,0xa0,0x47,0xb1,0xff,0xaf,0x55,0xc4],
                    "sbox_azam_3" : [
        0xb6,0xf9,0x75,0xc7,0x79,0x3a,0xc4,0xdc,0xac,0xf1,0x64,0xd0,0x17,0x01,0x24,0x00,
        0x4d,0x2a,0x57,0x0f,0x61,0x9a,0x81,0xd9,0x7d,0x42,0x1e,0x6b,0x6e,0x2e,0x21,0x5d,
        0x9e,0x4e,0x6c,0x47,0xa5,0xba,0x11,0xb7,0xea,0x82,0x7e,0xdd,0xdb,0x9b,0xed,0xcb,
        0x8a,0x89,0xf3,0x7b,0x95,0x9c,0x4a,0x60,0xbd,0xc8,0xd3,0xa2,0xaf,0x5c,0xf2,0x70,
        0xf4,0x78,0xa8,0x59,0xe3,0xc3,0x2f,0x32,0x13,0x4f,0x76,0x3b,0xdf,0x88,0xbb,0xaa,
        0x8f,0x31,0xc2,0xc5,0x53,0x12,0x37,0xcd,0xe4,0xb5,0x15,0xd7,0x66,0x92,0xa1,0xfc,
        0xef,0x96,0x14,0x33,0x9d,0x08,0x48,0x80,0x30,0x0a,0xfa,0x7c,0xe7,0x3e,0x43,0xc9,
        0xa0,0x16,0x2d,0x91,0x86,0xa4,0xd1,0x97,0xd5,0xab,0x7f,0x1c,0x7a,0x44,0xb2,0x71,
        0x35,0x46,0xad,0x9f,0xae,0x67,0x55,0x27,0xb9,0x74,0xe5,0x93,0x6d,0xcc,0xa3,0x72,
        0xe1,0x29,0x6a,0x65,0x8c,0x54,0xd6,0xb4,0xb3,0xa6,0x52,0x50,0xfe,0xbf,0xa9,0xe8,
        0xb8,0xe0,0x40,0xe2,0x10,0x1a,0x99,0x04,0xce,0xd2,0x5e,0x4b,0x38,0x07,0xc1,0xec,
        0xcf,0xde,0xda,0x06,0x5f,0x94,0x22,0x05,0x3d,0x39,0xe9,0x63,0xf8,0x98,0xd8,0x34,
        0x6f,0x77,0x4c,0x1d,0x68,0x3c,0xee,0x0e,0x5a,0x8b,0xf0,0x73,0x09,0x0d,0xbe,0x20,
        0xca,0xf7,0x0b,0xff,0x87,0xfd,0x0c,0x25,0x1f,0x19,0xe6,0x23,0xc6,0x8d,0xc0,0x2c,
        0x62,0xfb,0x83,0x36,0x1b,0x90,0x45,0xb1,0x26,0xf5,0x49,0x84,0xbc,0xeb,0x56,0x18,
        0x58,0x28,0x69,0xd4,0x2b,0xb0,0x51,0x02,0x3f,0x03,0x41,0x8e,0x85,0x5b,0xa7,0xf6]}
        st10 = ct[self.INVSHIFT_undo[bnum]]
        st9 = inv_sbox_m[sbox_type][ct[bnum] ^ guess[bnum]]
        return (st9 ^ st10)
    
    def process_known_key(self, inpkey):
        input_key = ""
        for subkey in inpkey:
            input_key = input_key + format(subkey, "02x")
        return AESpy.get_round_key(input_key, 10, sbox_type)

In [3]:
def format_stat(stat):
    return str("{:02X}<br>{:.3f}".format(stat[0], stat[2]))

def color_corr_key(row):
    global round_10_key
    ret = [""] * 16
    for i,bnum in enumerate(row):
        if bnum[0] == round_10_key[i]:
            ret[i] = "color: red"
        else:
            ret[i] = ""
    return ret
    
def check_success(bestguess):
    global successCount
    recv_key = AESpy.get_initial_key(bestguess, 10, sbox_type)
    if initial_key == recv_key:
        successCount +=1
        if successCount >= 10:
            return 1
        else:
            return 0
    else:
        successCount = 0
        return 0
    
class StopExecutionException(Exception):
    pass

def stats_callback():
    global current_trace_iteration
    global result_success
    global partialGuessingEntropy
    
    results = attack.results
    results.set_known_key(round_10_key)
    
    # Success rate calculation
    bestguess = [kguess[0][0] for kguess in results.find_maximums()]
    results_success[current_trace_iteration] = check_success(bestguess)
    if results_success[current_trace_iteration] == 1:
        raise StopExecutionException("Reached Limit")
    
    stat_data = results.find_maximums()
    df = pd.DataFrame(stat_data).transpose()
    clear_output(wait=True)
    tstart = current_trace_iteration * resolution
    tend = tstart + resolution
    current_trace_iteration += 1
    display(df.head().style.format(format_stat).apply(color_corr_key,axis=1).set_caption("Iteration {}. Trial {}. Finished traces {} to {}. Success {}".format(iteration, trial, tstart, tend, successCount)))

In [6]:
global current_trace_iteration
global result_success
global successCount

resolution = 25
max_n_traces = 4000
n_trials = 10
max_n_iteration = 20

for iteration in range(1,max_n_iteration+1):

    project_file = "../build/sca_success_rate_" + sbox_dict[sbox_type] + "/sca_test_CW305_pop_" + str(iteration) + ".cwp"
    project = cw.open_project(project_file)

    traces = project.waves[:]
    textin = project.textins[:]
    keys = project.keys[:]
    ciphertext = project.textouts[:]

    for trial in range(0,n_trials):

        success_file = "../build/sca_success_rate/sca_test_CW305.cwp"
        success_project = cw.create_project(success_file, overwrite=True)

        for n_trace in range(max_n_traces):
            traces_trial = traces[trial*max_n_traces+n_trace]
            textin_trial = textin[trial*max_n_traces+n_trace]
            ciphertext_trial = ciphertext[trial*max_n_traces+n_trace]
            keys_trial = keys[trial*max_n_traces+n_trace]
            trace_i = Trace(traces_trial, textin_trial, ciphertext_trial, keys_trial)
            success_project.traces.append(trace_i)

        success_project.save()

        print(len(success_project.traces[:]))

        successCount = 0
        current_trace_iteration = 0
        results_success = [1] * int(200)

        initial_key = list(success_project.keys[0])

        inpkey = success_project.keys[0]
        input_key = ""
        for subkey in inpkey:
            input_key = input_key + format(subkey, "02x")
        round_10_key = AESpy.get_round_key(input_key, 10, sbox_type)

        attack = cwa.cpa(success_project, cwa.leakage_models.new_model(AES128_Round10_Model))
        try:
            results = attack.run(stats_callback, resolution)
        except StopExecutionException as e:
            print(f"Execution stopped: {e}")

        success_project.close()

        # Success Rate saving
        success_file_name = "./results/sca_test_cpa_success_rate_10/" + sbox_type + "/success_results_trial_" + str(trial+10*(iteration-1)) + ".txt"
        with open(success_file_name,'w') as success_file:
            for success in results_success:
                print(success, file=success_file)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,820.137,2D0.102,6F0.119,1B0.096,D40.129,E70.119,6B0.116,980.134,850.122,760.110,360.151,180.143,C70.146,110.115,FB0.144,BE0.114
1,3D0.106,F20.087,B40.084,260.084,180.099,A10.093,960.095,8B0.082,6F0.088,ED0.094,F30.104,800.095,080.101,D00.085,EE0.089,FB0.104
2,620.091,9E0.087,3C0.083,240.084,FB0.091,7E0.084,E60.086,CB0.082,380.084,E60.090,5A0.090,320.087,250.090,6D0.081,180.088,0C0.093
3,440.090,D20.086,020.082,2E0.083,A20.090,570.083,E90.085,C50.082,570.084,410.089,1D0.089,680.081,390.086,8E0.080,270.084,3D0.085
4,810.089,ED0.085,4C0.081,DE0.081,800.088,010.083,F00.085,440.081,940.080,D80.087,AB0.088,BD0.079,D50.082,7B0.079,EF0.084,D60.082


Execution stopped: Reached Limit


In [6]:
success_rate = [0] * int(max_n_traces/resolution)

n_trials = 100

def mean(X):
    return np.sum(X, axis=0)/len(X)

trials_success_results_temp = []
for i in range(n_trials):
    success_file_name = "./results/sca_test_cpa_success_rate_10/" + sbox_type + "/success_results_trial_" + str(i) + ".txt"
    with open(success_file_name, 'r') as file_read:
        data = [int(line.strip()) for line in file_read]
        trials_success_results_temp.append(data)

trials_success_results = list(zip(*trials_success_results_temp))

for i in range(int(max_n_traces/resolution)):
    success_rate[i] = mean(trials_success_results[i])

In [7]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()
p = figure(plot_width=800)

xrange = range(len(success_rate))
p.line(xrange, success_rate, line_color="red")
show(p)

Loading BokehJS ...